#### Assignment 1: An Experimental Study on Treaps


Key components:

1) Data Generator

2) Randomized Treap

3) Competitior

In [1]:
import random

In [ ]:
class DataGenerator():
    def __init__(self):
        self.data = []
        self.id_next = 1

    def gen_element(self):
       # sample a uniform random integer key from [0, 10^7]
       key = random.randint(0, 10**7)
       id = self.id_next
       self.id_next += 1
       self.data.append(key)
       return (id, key)
    
    def gen_insertion(self):
        x = self.gen_element()
        return (1,x)
    
    def gen_deletion(self):
        # sample a uniform random integer id from [1, id_next-1]
        id_del = random.randint(1, self.id_next-1)
        # get the key associated with id_del
        key_del = self.data[id_del-1]
        return (2, key_del)

    def gen_search(self):
      # sample a uniform random integer key from [0, 10^7]
       key_sch = random.randint(0, 10**7)
       return (3, key_sch)  

       